## Categorical Data Encoding Techniques

Categorical variables can be of 3 types -

 1. **Binary variable** - Variable which has only two possible values.
                          eg. Male/Female , True/False, Y/N etc.
                          
 2. **Ordinal variable** - where some inherent order or sequence is present in variable.
                         eg. Grades - A,B,C or difficulty-level : simple, average,difficult etc.
                         
 3. **Nominal variable** - where no inherent order or no numerical importance present in that variable
                           eg. city, Department, Species etc.

In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder,OneHotEncoder,OrdinalEncoder

import category_encoders as ce

### Binary Feature Encoding

In [2]:
data=pd.DataFrame({'Col':['Y','N','Y','N','N','Y','Y','N','Y']})

In [3]:
data

,Col
0,Y
1,N
2,Y
3,N
4,N
5,Y
6,Y
7,N
8,Y


##### 1. Using Map

In [4]:
data['Col_map'] = data['Col'].map({'Y':1,'N':0})

##### 2. Using Replace

In [5]:
data['Col_replace'] = data['Col'].replace({'Y':1,'N':0})

In [6]:
data

,Col,Col_map,Col_replace
0,Y,1,1
1,N,0,0
2,Y,1,1
3,N,0,0
4,N,0,0
5,Y,1,1
6,Y,1,1
7,N,0,0
8,Y,1,1


### Ordinal Feature Encoding

In [7]:
data=pd.DataFrame({'Degree':['High school','Masters','Diploma','Bachelors','Bachelors','Masters','Phd','High school','High school']})

##### 1. Manual encoding ( Using map/replace )

In [8]:
dict_encode = {'None':0,'High school':1,'Diploma':2,'Bachelors':3,'Masters':4,'Phd':5}

data['Degree_map'] =data['Degree'].map(dict_encode)
data['Degree_replace'] =data['Degree'].replace(dict_encode)

##### 2. Using Label Encoder

In [9]:
lblencoder = LabelEncoder()
lblencoder.fit(data['Degree'])
data['Degree_lbl'] = lblencoder.transform(data['Degree'])

###### Drawbacks - 

1. It is supposed to be used for encoding of the target variable
2. prior to this missing value treatment is required
3. Need to make sure ordering is correct after label encoding.

##### 3. Using ordinal encoder

In [10]:
ord_encoder = OrdinalEncoder(dtype='int64')
ord_encoder.fit(data[['Degree']])
data['Degree_ord'] = ord_encoder.transform(data[['Degree']])

In [11]:
data

,Degree,Degree_map,Degree_replace,Degree_lbl,Degree_ord
0,High school,1,1,2,2
1,Masters,4,4,3,3
2,Diploma,2,2,1,1
3,Bachelors,3,3,0,0
4,Bachelors,3,3,0,0
5,Masters,4,4,3,3
6,Phd,5,5,4,4
7,High school,1,1,2,2
8,High school,1,1,2,2


### Nominal Feature Encoding

In [12]:
data=pd.DataFrame({'City':['Delhi','Mumbai','Hyderabad','Chennai','Bangalore','Delhi','Hyderabad','Mumbai','Agra']})

In [13]:
n=data['City'].nunique()
n

6

In [14]:
#data['City'].value_counts().index.tolist()

data['City'].value_counts()

Hyderabad    2
Mumbai       2
Delhi        2
Agra         1
Chennai      1
Bangalore    1
Name: City, dtype: int64

##### 1. Using One-hot encoder

In [15]:
ohe = OneHotEncoder(drop='first',sparse=False,dtype='int64')
ohe.fit(data[['City']])

data_ohe = pd.DataFrame(ohe.transform(data[['City']]))

data_ohe.columns = ['City_'+str(i) for i in range(n-1)]

data = pd.concat([data,data_ohe],axis=1)

data.drop(['City'],axis=1,inplace=True)

data

,City_0,City_1,City_2,City_3,City_4
0,0,0,1,0,0
1,0,0,0,0,1
2,0,0,0,1,0
3,0,1,0,0,0
4,1,0,0,0,0
5,0,0,1,0,0
6,0,0,0,1,0
7,0,0,0,0,1
8,0,0,0,0,0


##### 2. Using pandas get_dummies()

In [16]:
data=pd.DataFrame({'City':['Delhi','Mumbai','Hyderabad','Chennai','Bangalore','Delhi','Hyderabad','Mumbai','Agra']})

In [17]:
city_df = pd.get_dummies(data,drop_first=True,dtype='int64')  ## create dummy df and drop 1st dummy col to get n-1 cols

data = pd.concat([data,city_df],axis=1) ## concatenate dummy and original dataframe

data.drop(['City'],axis=1,inplace=True) ## drop original column

data

,City_Bangalore,City_Chennai,City_Delhi,City_Hyderabad,City_Mumbai
0,0,0,1,0,0
1,0,0,0,0,1
2,0,0,0,1,0
3,0,1,0,0,0
4,1,0,0,0,0
5,0,0,1,0,0
6,0,0,0,1,0
7,0,0,0,0,1
8,0,0,0,0,0


##### Drawbacks

1. If the high cardinality is present in the categorical feature then it leads to increase the dimensionality of dataset.
2. Dummy variable trap - feature are highly correlated.

##### 3. Using Binary encoder

Binary encoding is a combination of **Hash encoding and one-hot encoding**. 
In this encoding scheme, the categorical feature is first converted into numerical using an ordinal encoder. 
Then the numbers are transformed in the binary number. 
After that binary value is split into different columns.
Binary encoding works really well when there are a high number of categories

In [18]:
data=pd.DataFrame({'City':['Delhi','Mumbai','Hyderabad','Chennai','Bangalore','Delhi','Hyderabad','Mumbai','Agra']})

In [19]:
binencode = ce.BinaryEncoder(cols='City',return_df=True)
binencode.fit(data['City'])
binencode.transform(data['City'])

,City_0,City_1,City_2,City_3
0,0,0,0,1
1,0,0,1,0
2,0,0,1,1
3,0,1,0,0
4,0,1,0,1
5,0,0,0,1
6,0,0,1,1
7,0,0,1,0
8,0,1,1,0


##### Advantage

memory-efficient encoding scheme as it uses fewer features than one-hot encoding.

##### 4. Using Base N Encoder

In [20]:
baseN_encode = ce.BaseNEncoder(cols='City',base=5,return_df=True)

In [21]:
baseN_encode.fit(data['City'])

BaseNEncoder(base=5, cols=['City'], drop_invariant=False,
             handle_missing='value', handle_unknown='value',
             mapping=[{'col': 'City',
                       'mapping':     City_0  City_1  City_2
 1       0       0       1
 2       0       0       2
 3       0       0       3
 4       0       0       4
 5       0       1       0
 6       0       1       1
-1       0       0       0
-2       0       0       0}],
             return_df=True, verbose=0)

In [22]:
baseN_encode.transform(data['City'])

,City_0,City_1,City_2
0,0,0,1
1,0,0,2
2,0,0,3
3,0,0,4
4,0,1,0
5,0,0,1
6,0,0,3
7,0,0,2
8,0,1,1


##### Advantage

1. Reduces number of features requires to encode a categorical feature
2. Improves memory usage

##### 5. Using Hash encoding

Hash encoding tranforms arbitrary size input input fixed sized values

In [23]:
Hash_encode = ce.HashingEncoder(cols='City',return_df=True,n_components=4)

In [24]:
Hash_encode.fit(data['City'])

HashingEncoder(cols=['City'], drop_invariant=False, hash_method='md5',
               max_process=4, max_sample=2, n_components=4, return_df=True,
               verbose=0)

In [25]:
Hash_encode.transform(data['City'])

,col_0,col_1,col_2,col_3
0,0,0,1,0
1,0,1,0,0
2,1,0,0,0
3,0,0,1,0
4,1,0,0,0
5,0,0,1,0
6,1,0,0,0
7,0,1,0,0
8,0,0,1,0


##### Advantage 

Can represent data into lesser number of dimensions

##### Drwabacks

1. It may leads to loss of information.
2. **Collison** - When multiple numbers can be represent using same hash values

##### 6. Using Effect Encoding

Also known as sum encoding/Deviation encoding. <br>
Similiar to dummy encoding but it uses 0,1 and -1 to represent the categorical values

In [26]:
sum_encode = ce.SumEncoder(cols='City',return_df=True)
sum_encode.fit(data['City'])
sum_encode.transform(data['City'],)

,intercept,City_0,City_1,City_2,City_3,City_4
0,1,1.0,0.0,0.0,0.0,0.0
1,1,0.0,1.0,0.0,0.0,0.0
2,1,0.0,0.0,1.0,0.0,0.0
3,1,0.0,0.0,0.0,1.0,0.0
4,1,0.0,0.0,0.0,0.0,1.0
5,1,1.0,0.0,0.0,0.0,0.0
6,1,0.0,0.0,1.0,0.0,0.0
7,1,0.0,1.0,0.0,0.0,0.0
8,1,-1.0,-1.0,-1.0,-1.0,-1.0


##### 7. Using Target Encoding ( Bayesian Encoding)

In taget encoding, <br>
If target variable is **numerical** then -> Calculate mean of target for each category and replace category variable with mean.<br>
If target variable is **Categorical** then -> posterior probability of that target category replaces each category feature

In [27]:
data = pd.DataFrame({'Income':['H','M','L','M','L','H','H','H'],'Age':[50,30,65,75,53,88,80,60]})

## Income -> categorical variable
## Age -> Target variable

In [28]:
tgtencode = ce.TargetEncoder(cols='Income',return_df=True)
tgtencode.fit(data['Income'],data['Age'])
tgtencode.transform(data['Income'],data['Age'])


,Income
0,69.173947
1,55.223032
2,59.974913
3,55.223032
4,59.974913
5,69.173947
6,69.173947
7,69.173947


##### Categorical Feature are in form of bins

If categorical feature are present in form of bins then we can use below techniques to convert those into numerical feature:<br> <br>
1. Use label encoder and convert each bin (not useful)
2. Calculate mean/median for each bin and use that.
3. Create 2 new feature based on lower bound and upper bound of each bin

##### Combine Levels 

To get rid of the redundant levels within categorical feature, we can combine different levels using below :<br><br>

1. combine similar levels into same groups based on domain/ business knowledge.
2. combine based on frequency - levels are which have lesser freuency , combine them into same gorup.
3. combine based on response rate of each levels- combine level which has similiar response rates
4. combine based on both frequency and response rates - First combine similiar response rate levels then look for level which has very low freuency and combine them.